In [1]:
# import pandas, plotly
import pandas as pd
import plotly.express as px

In [2]:
# read in all csv files
aisles = pd.read_csv('aisles.csv')
departments = pd.read_csv('departments.csv')
order_prior = pd.read_csv('order_products__prior.csv')
order_train = pd.read_csv('order_products__train.csv')
orders = pd.read_csv('orders.csv')
products = pd.read_csv('products.csv')

In [18]:
# concatenate train and prior orders, order by order ID
pt = pd.concat([order_prior, order_train]).sort_values(by=['order_id'])

# merge pt with aisles, departments, orders, products
## keep pt on left for all merges, index by order ID 
pt = pd.merge(left = pt, right = products,
            left_on='product_id', right_on='product_id').sort_values(by=['order_id']).reset_index(drop=True)
pt = pd.merge(left = pt, right = aisles,
            left_on='aisle_id', right_on='aisle_id').sort_values(by=['order_id']).reset_index(drop=True)
pt = pd.merge(left = pt, right = departments,
            left_on='department_id', right_on='department_id').sort_values(by=['order_id']).reset_index(drop=True)
pt = pd.merge(left = pt, right = orders,
            left_on='order_id', right_on='order_id').sort_values(by=['order_id']).reset_index(drop=True)

# index to re-order columns in pt
cord = ['user_id', 'order_id', 'product_id', 'aisle_id', 'department_id', 'add_to_cart_order',
       'reordered', 'product_name', 'aisle', 'department', 'eval_set', 'order_number', 'order_dow',
       'order_hour_of_day', 'days_since_prior_order']

# re-order columns in pt
pt = pt[cord]

In [89]:
# head master df
pt.head()

,user_id,order_id,product_id,aisle_id,department_id,add_to_cart_order,reordered,product_name,aisle,department,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,112108,1,49302,120,16,1,1,Bulgarian Yogurt,yogurt,dairy eggs,train,4,4,10,9.0
1,112108,1,49683,83,4,4,0,Cucumber Kirby,fresh vegetables,produce,train,4,4,10,9.0
2,112108,1,13176,24,4,6,0,Bag of Organic Bananas,fresh fruits,produce,train,4,4,10,9.0
3,112108,1,43633,95,15,5,1,Lightly Smoked Sardines in Olive Oil,canned meat seafood,canned goods,train,4,4,10,9.0
4,112108,1,10246,83,4,3,0,Organic Celery Hearts,fresh vegetables,produce,train,4,4,10,9.0


In [25]:
# find how many orders each user placed
orders_placed = orders.groupby(['user_id'])['order_id'].agg(lambda group: len(group.unique()))\
                .reset_index(name='order_count').set_index('user_id')

In [42]:
# plot hist of orders placed per user using plotly
fig1 = px.histogram(orders_placed, x='order_count',
                   title='Order Frequency by User',
                   labels={'order_count':'Orders Placed'},
                   opacity=.8,
                   color_discrete_sequence=['darkorange'])
fig1.show()

In [166]:
fig1.write_html("total_orders.html")

In [58]:
# df of product frequency over all users/orders
prod_freq = pd.DataFrame(pt.product_name.value_counts() / pt.shape[0])
# drop index for plotting
prod_freq.reset_index(inplace=True)
# rename columns
prod_freq.columns = ['product_name', 'frequency']

In [161]:
# plot hist of products frequency using plotly
fig2 = px.histogram(prod_freq[:20], x='product_name', y='frequency',
                   title='Top 20 Products Order Frequency',
                   opacity=.8,
                   color_discrete_sequence=['forestgreen'],
                   height = 650, width = 800)

# update axis labels
fig2.update_layout(
    xaxis_title='Product Name',
    yaxis_title='Percent of Total Orders (%)')

fig2.show()

In [165]:
fig2.write_html("product_frequency.html")

In [116]:
# df for order frequency by dow and hodb
order_freq = orders.groupby(['order_dow', 'order_hour_of_day'])['order_number'].aggregate('count').reset_index()

In [117]:
# convert wide to long data for plotting
order_freq_l = order_freq.pivot(index = 'order_dow', columns = 'order_hour_of_day', values = 'order_number')

In [163]:
# create heatmap of order frequency by day and hour
fig3 = px.imshow(order_freq_l,
                 title=('Order Frequency by Day and Hour'),
                 y = ['Sunday','Monday','Tuesday','Wednesday','Thursday','Friday','Saturday'],
                 x = ['12AM', '1AM', '2AM', '3AM', '4AM', '5AM', '6AM', '7AM', '8AM',
                     '9AM', '10AM', '11AM', '12PM', '1PM', '2PM', '3PM', '4PM', '5PM',
                     '6PM', '7PM', '8PM', '9PM', '10PM', '11PM'],
                 labels = dict(x='Time of Day', y='Day of the Week', color='# of Orders'),
                 color_continuous_scale='Geyser',
                 aspect = 2)
fig3.show()

In [164]:
fig3.write_html("time_heatmap.html")

In [170]:
# df of frequency of each department
dep_freq = pd.DataFrame(pt.department.value_counts() / pt.shape[0])
# drop index for plotting
dep_freq.reset_index(inplace=True)
# rename columns
dep_freq.columns = ['department', 'frequency']

In [182]:
fig4 = px.pie(dep_freq, values = 'frequency', names = 'department',
             title = 'Purchases by Department',
             color_discrete_sequence=px.colors.qualitative.Prism)

fig4.show()

In [183]:
fig4.write_html('dep_purchases.html')